In [43]:
import xlrd
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
import plotly as py
import numpy as np
import folium
from folium import plugins
from IPython.display import HTML, display
from folium.plugins import HeatMap


In [92]:
file_location = pd.read_csv("./youth_tobacco_analysis.csv")
data = pd.DataFrame(file_location)
#print(data['GeoLocation'])
data = pd.DataFrame(data.groupby(['GeoLocation'])['Sample_Size'].agg(['sum']))
data.to_csv('./question6.csv', sep = ',', encoding='utf-8')
data = pd.read_csv('./question6.csv')
lat = []
lon = []
data = data.dropna(subset=['GeoLocation','sum'])
data['GeoLocation'] = data['GeoLocation'].str.strip('()')
for row in data['GeoLocation']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)


# Create two new columns from lat and lon
data['latitude'] = lat
data['latitude'] = data['latitude'].astype(float)
data['longitude'] = lon
data['longitude'] = data['longitude'].astype(float)
max_amount = data['sum'].max()
values = []
for i in data['sum']:
    values.append(i)

url_base = 'http://server.arcgisonline.com/ArcGIS/rest/services/'
service = 'NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
tileset = url_base + service
colors = {'A' : 'red', 'B' : 'blue'}

hmap = folium.Map(location=[data.latitude[10], data.longitude[10]], zoom_start=3,control_scale = True, tiles=tileset, attr='USGS style' )

hm_wide = HeatMap( list(zip(data.latitude, data.longitude, values)), 
                   min_opacity=0.2,
                   max_val=max_amount,
                   radius=17, blur=10, 
                   max_zoom=1, 
                 )

hmap.add_child(hm_wide)